# GPT Prompt 생성


> **주제: 한국 보험 약관의 해석 및 비판적 분석** <br/><br/>
Topic: Interpretation and critical analysis of Korean insurance terms

## Step 3: 문제점 '평가' 및 중요 문제 선정 (ToT 평가, RAG 활용 가능)

### 목표: 발상된 문제점 후보들을 중요도 기준으로 평가하고 최적을 선정하는 것. 평가 기준을 판단할 때 RAG로 추가 정보를 얻으면 더 객관적인 평가 가능.
### 수정 내용:
* 문제점 후보들을 평가할 때, 각 문제점과 관련된 약관 원문 및 Step 2에서 문제점 발상 시 사용되었던 외부 참고 자료와 함께 GPT 프롬프트에 넣음

* 프롬프트 지침에 "제시된 문제점 후보와 관련 약관 및 **[참고 자료]**를 바탕으로 심각성, 발생 가능성, 모호성 등을 평가하라"고 명시
* 평가 기준(심각성, 발생 가능성 등)을 판단할 때 RAG로 검색된 새로운 외부 정보 (예: 특정 문제점 유형에 대한 최근 통계, 최신 법원 판례 등)를 추가로 제공하여 평가의 정확성을 높일 수 있음 (평가 단계마다 검색해야 해서 복잡해질 수 있음)

* 평가 결과를 JSON으로 받고, 총점 기준으로 순위 매겨 최적 문제점을 선정하는 것은 동일.
----

### basic setting

In [2]:
!pip install -U langchain-community langchain-openai chromadb PyMuPDF langchain-chroma

In [3]:
# 필요한 라이브러리 임포트
import os
import json
import re
from google.colab import drive
drive.mount('/content/drive')
from google.colab import userdata
from collections import defaultdict # 평가 함수 등에 필요

Mounted at /content/drive


In [4]:
# Langchain 관련 임포트 (최신 langchain-chroma 및 langchain-openai 사용)
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma # <-- 최신 임포트 경로 확인!

from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.documents import Document

In [5]:
print("OpenAI API 키 설정 확인 중 (userdata 사용)")
openai_api_key = userdata.get('OPENAI_API_KEY')

OpenAI API 키 설정 확인 중 (userdata 사용)


In [6]:
openai_api_key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = xx


In [7]:
import openai
client = openai.OpenAI(api_key = userdata.get('OPENAI_API_KEY'))

In [8]:
import glob
import tiktoken

In [9]:
if openai_api_key is None:
    print("!!! 오류: Colab '비밀'에서 'OPENAI_API_KEY'를 가져오지 못했습니다. !!!")
    print("이 노트북에서도 Colab 왼쪽의 자물쇠 아이콘(비밀)에서 'OPENAI_API_KEY' 이름으로 키를 설정해주세요.")
    print("API 키 없이는 임베딩 모델 로드 및 GPT 모델 호출이 불가능합니다.")
    # API 키 없이는 임베딩 모델 로드 및 GPT 호출이 불가능하므로 관련 객체는 None
    embeddings = None
    llm = None
    rag_vectorstore = None # 로드 불가
else:
    print("OpenAI API 키 설정 완료.")

OpenAI API 키 설정 완료.


In [10]:
# 임베딩 모델 로드 (벡터 스토어 로드 시 필요)
# Step 0에서 사용했던 모델과 동일해야 합니다.
try:
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    print("OpenAI 임베딩 모델 준비 완료.")
except Exception as e:
    print(f"!!! 오류: 임베딩 모델 로드 오류: {e} !!!")
    print("환경 변수에 설정된 API 키가 유효한지, 모델 이름이 올바른지 확인하세요.")
    embeddings = None

OpenAI 임베딩 모델 준비 완료.


In [ ]:
# GPT 모델 준비 (평가 시 필요)
try:
    llm = ChatOpenAI(model="gpt-4o-2024-05-13", temperature=0.0) # 평가 단계는 temperature 낮게
    print(f"GPT 모델 준비 완료: {llm.model_name}")
except Exception as e:
    print(f"!!! 오류: GPT 모델 로드 오류: {e} !!!")
    print("환경 변수에 설정된 API 키가 유효한지, 모델 이름이 올바른지 확인하세요.")
    llm = None

GPT 모델 준비 완료: gpt-4o-2024-05-13


#### step2 결과 불러오기

In [11]:
# --- Step 2 결과 JSON 파일 경로 지정 ---
# Step 2 노트북에서 결과를 저장했던 그 경로와 파일 이름과 동일해야 합니다.
step2_results_save_dir = '/content/drive/MyDrive/PROJECT/cn_api_project/step2_results' # 예시 경로


In [12]:
print(f"\n--- Step 2 결과 JSON 파일들 로드 중 (디렉토리: '{step2_results_save_dir}') ---")
# 모든 문제점 후보 데이터를 담을 리스트
all_loaded_problem_candidates = []
# 디렉토리 순회
if os.path.exists(step2_results_save_dir):
    json_files = [f for f in os.listdir(step2_results_save_dir) if f.endswith('.json')]

    if not json_files:
        print("⚠️ JSON 파일이 존재하지 않습니다.")
    else:
        for json_file in json_files:
            file_path = os.path.join(step2_results_save_dir, json_file)
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    all_loaded_problem_candidates.extend(data)

                print(f"✅ '{json_file}' 로드 완료 - 문제점 후보 수: {len(data)}")

            except Exception as e:
                print(f"❌ '{json_file}' 로드 실패 - 오류: {e}")
else:
    print(f"❌ 디렉토리를 찾을 수 없습니다: '{step2_results_save_dir}'")

print(f"\n총 로드된 문제점 후보 개수: {len(all_loaded_problem_candidates)}")



--- Step 2 결과 JSON 파일들 로드 중 (디렉토리: '/content/drive/MyDrive/PROJECT/cn_api_project/step2_results') ---
✅ 'problems_한화_스마일펫보험.json' 로드 완료 - 문제점 후보 수: 227
✅ 'problems_삼성_다이렉트_실손보험.json' 로드 완료 - 문제점 후보 수: 442
✅ 'problems_삼성화재_반려묘보험_의기냥냥.json' 로드 완료 - 문제점 후보 수: 308

총 로드된 문제점 후보 개수: 977


In [13]:
def load_problem_candidates(step2_results_folder):
    problem_candidates = []
    for json_file in glob.glob(os.path.join(step2_results_folder, "*.json")):
        with open(json_file, "r", encoding="utf-8") as f:
            data = json.load(f)
            if isinstance(data, list):
                problem_candidates.extend(data)
            else:
                problem_candidates.append(data)
    print(f"✅ 문제점 후보 {len(problem_candidates)}개 로딩 완료")
    return problem_candidates

def load_summary_and_original_text(summary_file_path, terms_folder):
    with open(summary_file_path, "r", encoding="utf-8") as f:
        summary_text = f.read()
    common_prefix = os.path.basename(summary_file_path).replace("_summary.txt", "")
    original_file_path = os.path.join(terms_folder, f"{common_prefix}.txt")

    if os.path.exists(original_file_path):
        with open(original_file_path, "r", encoding="utf-8") as f:
            original_text = f.read()
        print(f"✅ 약관 원문 로딩 성공: {original_file_path}")
    else:
        raise FileNotFoundError(f"❌ 약관 원문 파일 없음: {original_file_path}")

    return summary_text, original_text, os.path.basename(summary_file_path)

# 텍스트를 토큰 기준으로 n개 이하로 분할하는 유틸 함수
def split_text_by_token_limit(text, token_limit=1500, model="gpt-4"):
    enc = tiktoken.encoding_for_model(model)
    tokens = enc.encode(text)

    chunks = []
    for i in range(0, len(tokens), token_limit):
        chunk_tokens = tokens[i:i+token_limit]
        chunk_text = enc.decode(chunk_tokens)
        chunks.append(chunk_text)

    print(f"✅ 텍스트 분할 완료: 총 {len(chunks)}개 청크 생성 (청크당 {token_limit}토큰 이하)")
    return chunks

In [ ]:
# 1. 변수 정의
step2_results_folder = "/content/drive/MyDrive/PROJECT/cn_api_project/step2_results"
summary_file_path = "/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/summary/한화_스마일펫보험_summary.txt"
terms_folder = "/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관"
save_folder = "/content/drive/MyDrive/PROJECT/cn_api_project/step3_results"

# 폴더 없으면 생성
os.makedirs(save_folder, exist_ok=True)

In [ ]:
# 2. 문제점 후보, 요약본, 약관 원문 로딩
problem_candidates_for_evaluation = load_problem_candidates(step2_results_folder)
previous_full_summary_text, analysis_target_text_full, original_filename = load_summary_and_original_text(summary_file_path, terms_folder)


# 데이터 로딩
problem_candidates = load_problem_candidates(step2_results_folder)
summary_text, original_text, summary_filename = load_summary_and_original_text(summary_file_path, terms_folder)


✅ 문제점 후보 669개 로딩 완료
✅ 약관 원문 로딩 성공: /content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/한화_스마일펫보험.txt
✅ 문제점 후보 669개 로딩 완료
✅ 약관 원문 로딩 성공: /content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/한화_스마일펫보험.txt


In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
# 3. 분할 처리 예시 (요약, 원문 각각 토큰 1500 기준으로 분할)
summary_chunks = split_text_by_token_limit(previous_full_summary_text, token_limit=1000)
terms_chunks = split_text_by_token_limit(analysis_target_text_full, token_limit=1000)


✅ 텍스트 분할 완료: 총 60개 청크 생성 (청크당 1000토큰 이하)
✅ 텍스트 분할 완료: 총 55개 청크 생성 (청크당 1000토큰 이하)


In [ ]:
# 텍스트 분할 (필요시)
summary_chunks = split_text_by_token_limit(summary_text, token_limit=1000)
original_chunks = split_text_by_token_limit(original_text, token_limit=1000)


✅ 텍스트 분할 완료: 총 60개 청크 생성 (청크당 1000토큰 이하)
✅ 텍스트 분할 완료: 총 55개 청크 생성 (청크당 1000토큰 이하)


In [19]:
final_evaluation_results = []

In [20]:
# 4. 문제점 후보를 수량 기준 분할 (예: 50개씩)
def split_list(lst, chunk_size):
    return [lst[i:i+chunk_size] for i in range(0, len(lst), chunk_size)]

In [21]:
# --- 벡터 스토어 영구 저장 경로 지정 ---
# 첫 번째 노트북에서 벡터 스토어를 저장했던 그 경로와 동일해야 합니다.
PERSIST_DIRECTORY = '/content/drive/MyDrive/PROJECT/cn_api_project/chroma_db' # 예시 경로

# --- 벡터 스토어 로드 ---
rag_vectorstore = None # 초기화
# 임베딩 모델이 성공적으로 로드되었고 (API 키가 설정되었고) 저장 경로가 존재하면 로드 시도
if embeddings is not None and os.path.exists(PERSIST_DIRECTORY):
    print(f"\n--- 벡터 스토어 로드 중: '{PERSIST_DIRECTORY}' ---")
    try:
        rag_vectorstore = Chroma(persist_directory=PERSIST_DIRECTORY, embedding_function=embeddings)
        print("벡터 스토어 로드 완료.")
        print(f"로드된 청크 개수: {rag_vectorstore._collection.count()}") # 로드된 청크 개수 확인!

    except Exception as e:
        print(f"!!! 오류: 벡터 스토어 로드 중 오류 발생 !!!")
        print(f"오류 내용: {e}")
        print(f"경로 확인: '{PERSIST_DIRECTORY}'에 파일 있는지, 로드 시 사용한 임베딩 모델이 올바른지 확인하세요.")
        rag_vectorstore = None
else:
    if embeddings is None:
        print("\n!!! 벡터 스토어 로드 불가: 임베딩 모델 로드 오류 (API 키 확인 필요). !!!")
    elif not os.path.exists(PERSIST_DIRECTORY):
         print(f"\n!!! 벡터 스토어 로드 불가: 저장 폴더를 찾을 수 없습니다 - '{PERSIST_DIRECTORY}' !!!")
         print("첫 번째 노트북에서 Step 0을 실행하여 벡터 스토어를 해당 경로에 저장했는지 확인해주세요.")


--- 벡터 스토어 로드 중: '/content/drive/MyDrive/PROJECT/cn_api_project/chroma_db' ---
벡터 스토어 로드 완료.
로드된 청크 개수: 37249


### 잠재적 문제점 평가 & 분석

#### 프롬프트 템플릿
> ToT 평가 + RAG Context 포함

In [18]:
# --- Step 3 - 문제점 평가 프롬프트 템플릿 (ToT 평가 + RAG Context 포함) ---
evaluation_template_problem_rag = """
## 역할: 소비자 보호 관점에서 보험 약관의 잠재적 문제점을 평가하는 전문 분석가

## 임무:
아래 **[잠재적 문제점 후보들]** 목록을 **[분석 대상 약관 원문 텍스트]**, **[이전 약관 전체 요약]**, 그리고 **[추가 참고 자료]**를 모두 종합하여 평가하고, 각 문제점 후보가 얼마나 중요하고 현실적인 문제인지 아래 JSON 형식으로 평가 결과를 반환해 주세요. 가장 중요하고 시급히 개선해야 할 문제점을 선별하기 위한 객관적인 평가를 수행합니다.
이 평가는 보험 약관의 개선 방향을 결정하기 위한 중요한 작업이며, **소비자 피해를 예방하고 분쟁을 줄이기 위한 목적**을 가지고 있습니다.

[잠재적 문제점 후보들 (JSON 문자열)]:
{problem_candidates_json_string}

[분석 대상 약관 원문 텍스트]:
{analysis_target_text}

[이전 약관 전체 요약]:
{previous_full_summary}

[추가 참고 자료]:
{rag_context}

## 평가 기준 (각 기준별로 10점 만점 부여):
1.  **문제의 심각성 (소비자 피해 규모):**
    *   이 문제로 인해 소비자가 입을 수 있는 피해가 얼마나 큰가요? (예: 전액 보험금 미지급, 계약 해지 등일수록 고득점)
    *   이 문제점이 실제로 발생했을 때 소비자가 입을 수 있는 **잠재적 피해의 크기**를 평가합니다. (예: 보험금 전액 미지급, 거액의 손실, 계약 해지 등 심각한 피해일수록 높은 점수)
    *   단순한 불편함이나 오해를 넘어 실질적인 재산상/법적 손실 가능성을 중점적으로 고려합니다.
2.  **실제 발생 가능성:**
    *   해당 문제점 시나리오가 **현실에서 실제로 발생할 가능성이 얼마나 높은지**를 평가합니다. (예: 약관 문구가 모호하여 자주 분쟁이 발생하는 유형이거나, 소비자가 흔히 겪는 상황일수록 높은 점)
    *   RAG 참고자료(분쟁사례, 과거 판례 등)를 적극 반영하여 실제 사례가 있으면 높게 평가하세요.
    *   [추가 참고 자료]의 분쟁 사례나 피해 사례 등을 참고하여 과거에 유사한 문제가 실제로 발생한 적이 있는지 고려합니다. 법규 위반 가능성이 명확할수록 발생 가능성이 높다고 봅니다.
3.  **약관 표현의 모호성 정도:**
    *   약관 문구가 얼마나 불명확한가요?
    *   해당 문제점을 유발하는 약관 문구나 조건이 **얼마나 불명확하고 여러 가지로 해석될 여지가 많은지**를 평가합니다. (예: 특정 용어 정의가 없거나 불분명한 경우, 보상/면책 조건이 애매모호한 경우 등 높은 점)
    *   명확하게 숫자로 정해져 있거나 예시가 잘 제시된 경우는 모호성이 낮다고 봅니다. 모호성이 높을수록 소비자의 오해나 보험사의 자의적 해석으로 이어질 가능성이 큽니다.

## 지침:
1.  **후보 목록 검토:** [잠재적 문제점 후보들] 목록의 각 문제점 요약, 시나리오, 근거_및_추론과정, 관련 약관 표현 등을 꼼꼼히 읽어 각 문제점의 내용을 정확히 이해합니다.
2.  **정보 종합 및 평가:** 각 문제점 후보에 대해 위에 제시된 3가지 **평가 기준**을 적용하여 평가를 수행합니다. **[분석 대상 약관 원문 텍스트]**에서 해당 문제점과 관련된 정확한 문구를 다시 확인하고, **[이전 약관 전체 요약]**으로 전체 맥락을 고려하며, **특히 [추가 참고 자료]**에 있는 분쟁 사례, 법규, 유사 약관 내용 등을 적극적으로 참고하여 각 기준별 점수를 **논리적으로** 결정합니다. (예: "참고 자료의 금감원 사례를 보니 이 문제로 실제 피해 본 사람이 많아 발생 가능성이 높다고 판단.")
3.  **평가 이유 요약 (CoT 평가):** 각 문제점 후보에 대해 점수를 부여한 **구체적인 이유**를 '평가_이유_요약' 항목에 간략하게 설명합니다. **왜 그 점수를 주었는지, 어떤 약관 문구나 참고 자료가 판단의 근거가 되었는지**를 명확히 밝힙니다.
                                각 점수에 대한 **구체적 이유**를 `평가_이유_요약`에 작성하세요.
                                    - 예: “금감원 사례집에서 유사 피해 사례가 반복 발생하여 발생 가능성 높음”
                                    - 약관 문구가 판단에 어떤 영향을 주었는지도 명시
4.  **JSON 출력 형식:** 모든 문제점 후보에 대한 평가 결과를 아래 JSON 형식의 **리스트**로 반환합니다. 각 결과 항목의 'id'는 평가 대상 문제점 후보의 'id'와 **반드시 일치**해야 합니다. 각 평가 기준별 점수(정수형)와 총점(30점 만점, 자동 합산)을 정확히 포함합니다.
5.  **평가 불가능 후보 처리:** 만약 정보 부족 등의 이유로 특정 문제점 후보를 평가하기 어렵다면, 해당 후보에 대해 모든 평가 기준에 **0점**을 부여하고 '평가_이유_요약'에 '정보 부족' 등으로 명시합니다.

## 출력 형식:
JSON

## 출력 시작:
```json
{{
    "평가_결과": [
        {{
            "id": <평가 대상 문제점 후보의 고유 ID (정수)>,
            "문제점_요약": "<평가 대상 문제점 요약 (원본 그대로 또는 간략히). 확인용.>",
            "문제의_심각성": <점수 (0-10)>,
            "실제_발생_가능성": <점수 (0-10)>,
            "약관_표현의_모호성_정도": <점수 (0-10)>,
            "총점": <세 기준 점수의 합 (0-30)>,
            "평가_이유_요약": "<이 문제점에 대해 점수를 부여한 구체적인 이유. 어떤 약관 문구/참고 자료를 보고 이렇게 판단했는지 간략히 설명.>"
        }},
        // ... 발상된 모든 문제점 후보에 대한 평가 결과 객체 추가 ...
    ]
}}
"""

#### request_gpt 헬퍼 함수

In [ ]:
def request_gpt(message, model="gpt-4o-2024-05-13", temperature=0.7, type="json_object"):
# 이 함수는 openai.OpenAI() 클라이언트 객체를 사용합니다.
# client 객체가 위에 정의되어 있다고 가정합니다.
    if client is None:
        print("API 클라이언트가 초기화되지 않았습니다.")
        return {} if type == "json_object" else "API 클라이언트 오류"

    messages = [{"role": "user", "content": message}]
    try:
        response = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=temperature,
                response_format={"type" : type}
            ).model_dump()
        content = response['choices'][0]['message']['content']
        if type == "json_object":
            content = content.strip()
            # 가끔 ```json ... ``` 형태로 오는 응답 처리
            if content.startswith("```json"): content = content[7:]
            if content.endswith("```"): content = content[:-3]
            content = content.strip()
            return json.loads(content)
        else: return content
    except Exception as e:
        print(f"API 호출 중 오류 발생: {e}")
        return {} if type == "json_object" else f"API Error: {e}"

#### evaluate_problems_with_rag 함수 정의

In [37]:
def evaluate_problems_with_rag(problem_candidates, analysis_target_text_full, previous_full_summary_text, rag_vectorstore, model="gpt-4o-2024-05-13"):
# RAG를 사용하여 발상된 문제점 후보들을 평가합니다. 약관/요약 전체 텍스트를 청크로 분할하여 프롬프트에 포함합니다.
# 이 함수에서는 request_gpt 헬퍼 함수를 사용합니다.
    if client is None:
        print("\n!!! API 클라이언트가 초기화되지 않아 문제점 평가를 실행할 수 없습니다. !!!")
        return []

    if not problem_candidates:
    print("\n--- [Step 3] 문제점 평가 (평가할 후보 없음) ---")
    return []

    # 평가 단계는 논리적인 평가를 위해 temperature를 낮게 설정 (0.0)
    evaluation_temperature = 0.0

    print(f"\n--- [Step 3] 잠재적 문제점 평가 ({len(problem_candidates)}개 후보 대상, RAG 활용 가능) ---")

## step3 실행

#### 변수 다시 정의

In [15]:
import os
import glob
import json

# 1. 문제점 후보들 로딩
step2_results_folder = "/content/drive/MyDrive/PROJECT/cn_api_project/step2_results"
problem_candidates_for_evaluation = []

for json_file in glob.glob(os.path.join(step2_results_folder, "*.json")):
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)
        # 파일 하나당 하나의 문제점 또는 복수 문제점 포함 가능
        if isinstance(data, list):
            problem_candidates_for_evaluation.extend(data)
        else:
            problem_candidates_for_evaluation.append(data)

print(f"✅ 문제점 후보 {len(problem_candidates_for_evaluation)}개 로딩 완료")

# 2. 요약본 텍스트 로딩
summary_file_path = "/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/summary/한화_스마일펫보험_summary.txt"
with open(summary_file_path, "r", encoding="utf-8") as f:
    previous_full_summary_text = f.read()

# 3. 약관 원문 텍스트 로딩 (파일명 접두어로 찾기)
common_prefix = os.path.basename(summary_file_path).replace("_summary.txt", "")
terms_folder = "/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관"
original_file_path = os.path.join(terms_folder, f"{common_prefix}.txt")

if os.path.exists(original_file_path):
    with open(original_file_path, "r", encoding="utf-8") as f:
        analysis_target_text_full = f.read()
    print(f"✅ 약관 원문 로딩 성공: {original_file_path}")
else:
    raise FileNotFoundError(f"❌ 약관 원문 파일 없음: {original_file_path}")

# 결과 확인
print("\n[입력 변수 확인]")
print(f"- 문제점 후보 수: {len(problem_candidates_for_evaluation)}")
print(f"- 요약문 길이: {len(previous_full_summary_text)}자")
print(f"- 원문 길이: {len(analysis_target_text_full)}자")

✅ 문제점 후보 977개 로딩 완료
✅ 약관 원문 로딩 성공: /content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/한화_스마일펫보험.txt

[입력 변수 확인]
- 문제점 후보 수: 977
- 요약문 길이: 65362자
- 원문 길이: 57259자


In [34]:
 # 약관 원문 전체 및 이전 요약 전체 텍스트 로드
analysis_target_text_full = load_text_file(TERMS_ORIGINAL_FOLDER)
previous_full_summary_text = load_text_file(summary_file_path)

if not analysis_target_text_full:
    print(f"❌ 오류: 해당 원본 약관 파일 로드 실패. 이 파일의 평가는 건너뜁니다.")


    if not previous_full_summary_text:
        print(f"!!! 경고: 해당 약관 요약본 파일 로드 실패. 이전 요약 없이 평가를 진행합니다.")
        # 이전 요약 없더라도 평가 진행 가능하므로 continue 대신 경고만 출력


loaded_problem_candidates = []
try:
    with open(step2_result_file_path, 'r', encoding='utf-8') as f:
        loaded_problem_candidates = json.load(f)
    print(f"  - Step 2 결과 로드 완료. 문제점 후보 {len(loaded_problem_candidates)}개.")
except Exception as e:
    print(f"❌ 오류: Step 2 결과 파일 로드 또는 파싱 중 오류 발생 ({filename}): {e}")
    loaded_problem_candidates = []


❌ 오류: 해당 원본 약관 파일 로드 실패. 이 파일의 평가는 건너뜁니다.
!!! 경고: 해당 약관 요약본 파일 로드 실패. 이전 요약 없이 평가를 진행합니다.
  - Step 2 결과 로드 완료. 문제점 후보 227개.


In [23]:
STEP3_RESULTS_SAVE_DIR = "/content/drive/MyDrive/PROJECT/cn_api_project/step3_results"
STEP2_RESULTS_DIR = "/content/drive/MyDrive/PROJECT/cn_api_project/step2_results"
TERMS_ORIGINAL_FOLDER = '/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관'
SUMMARY_FOLDER = '/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/summary'

In [25]:
from tqdm.notebook import tqdm

In [26]:
if STEP3_RESULTS_SAVE_DIR: # 평가 결과 저장 폴더가 유효하면 진행
    print(f"\n--- Step 2 결과 파일 순회 및 Step 3 평가 시작: '{STEP2_RESULTS_DIR}' ---")

    json_files_in_step2 = [f for f in os.listdir(STEP2_RESULTS_DIR) if f.lower().endswith('.json')]

    if not json_files_in_step2:
        print(f"\n경고: '{STEP2_RESULTS_DIR}' 폴더에 .json 파일이 없습니다. 평가를 진행할 수 없습니다.")

    for filename in tqdm(json_files_in_step2, desc="파일별 문제점 평가 진행"):
        step2_result_file_path = os.path.join(STEP2_RESULTS_DIR, filename)

        # --- Step 2 결과 파일 이름에서 원본 파일 이름 추출 ---
        name_part_with_summary = filename.replace(".json", "")
        name_part_base = name_part_with_summary.replace("_problem_candidates", "")
        name_part_base = name_part_base.replace("_summary", "")
        name_part_base = name_part_base.replace("_problems", "") # 혹시 problems 패턴도 있다면 추가


        # --- 해당 원본 약관 원문 텍스트 파일 경로 구성 및 로드 ---
        original_terms_filename = f"{name_part_base}.txt"
        original_terms_file_path = os.path.join(TERMS_ORIGINAL_FOLDER, original_terms_filename)

        # --- 해당 약관 요약본 텍스트 파일 경로 구성 및 로드 ---
        summary_filename = f"{name_part_base}_summary.txt"
        summary_file_path = os.path.join(SUMMARY_FOLDER, summary_filename)


        print(f"\n\n📄 평가 대상 파일: '{filename}'. 해당 원본 약관: '{original_terms_filename}'. 해당 요약본: '{summary_filename}'")

        # 약관 원문 전체 및 이전 요약 전체 텍스트 로드
        analysis_target_text_full = load_text_file(TERMS_ORIGINAL_FOLDER)
        previous_full_summary_text = load_text_file(SUMMARY_FOLDER)

        if not analysis_target_text_full:
             print(f"❌ 오류: 해당 원본 약관 파일 로드 실패. 이 파일의 평가는 건너뜁니다.")
             continue

        if not previous_full_summary_text:
             print(f"!!! 경고: 해당 약관 요약본 파일 로드 실패. 이전 요약 없이 평가를 진행합니다.")
             # 이전 요약 없더라도 평가 진행 가능하므로 continue 대신 경고만 출력


        # --- Step 2 결과 (문제점 후보 목록) 로드 ---
        loaded_problem_candidates = []
        try:
            with open(step2_result_file_path, 'r', encoding='utf-8') as f:
                loaded_problem_candidates = json.load(f)
            print(f"  - Step 2 결과 로드 완료. 문제점 후보 {len(loaded_problem_candidates)}개.")
        except Exception as e:
            print(f"❌ 오류: Step 2 결과 파일 로드 또는 파싱 중 오류 발생 ({filename}): {e}")
            loaded_problem_candidates = []


        # --- Step 3 문제점 평가 실행 ---
        if loaded_problem_candidates:
            print("  - 문제점 평가 함수 호출 중...")
            # evaluate_problems_with_rag 함수 호출
            # 로드된 문제점 후보 리스트, 약관 원문 전체, 이전 요약 전체, 벡터 스토어 전달
            evaluated_problem_results = evaluate_problems_with_rag(
                problem_candidates=loaded_problem_candidates, # 평가 대상 문제점 후보 목록 전달
                analysis_target_text_full=analysis_target_text_full, # 약관 원문 전체 전달
                previous_full_summary_text=previous_full_summary_text, # 이전 요약 전체 전달
                rag_vectorstore=rag_vectorstore, # 로드된 벡터 스토어 전달 (평가 시 RAG 활용)
                model="gpt-4o-2024-05-13" # GPT 모델
            )

            print(f"  - 문제점 평가 함수 실행 완료. 평가 결과 {len(evaluated_problem_results)}개.")

            # --- 평가 결과 저장 파일 이름 자동 생성 및 저장 ---
            if evaluated_problem_results:
                # 저장될 파일 이름 자동 생성 (evaluation_ + 원본이름 + .json)
                evaluation_filename = f"evaluation_{name_part_base}.json" # 예: "evaluation_한화_스마일펫보험.json"
                save_path = os.path.join(STEP3_RESULTS_SAVE_DIR, evaluation_filename) # 저장 경로

                print(f"  - 평가 결과 JSON 파일 저장 중: '{evaluation_filename}'")
                try:
                    with open(save_path, 'w', encoding='utf-8') as out_file:
                        json.dump(evaluated_problem_results, out_file, indent=2, ensure_ascii=False)

                    print(f"✅ 저장 완료: '{evaluation_filename}'")

                except IOError as e:
                    print(f"❌ 파일 저장 오류 ({evaluation_filename}): {e}")
                    print(f"경로 확인: '{save_path}' 경로에 쓸 수 있는지, Google Drive 마운트 상태가 올바른지 확인하세요.")

            else:
                print(f"경고: '{filename}' 파일에 대해 생성된 평가 결과가 없습니다.")

        else:
            print(f"경고: '{filename}' 파일에서 로드된 문제점 후보가 없어 평가를 건너뜁니다.")

print("\n--- 파일 순회 완료 ---")



--- Step 2 결과 파일 순회 및 Step 3 평가 시작: '/content/drive/MyDrive/PROJECT/cn_api_project/step2_results' ---


파일별 문제점 평가 진행:   0%|          | 0/3 [00:00<?, ?it/s]



📄 평가 대상 파일: 'problems_한화_스마일펫보험.json'. 해당 원본 약관: 'problems_한화_스마일펫보험.txt'. 해당 요약본: 'problems_한화_스마일펫보험_summary.txt'


NameError: name 'load_text_file' is not defined

In [ ]:
from  tqdm.notebook import tqdm
# 평가 실행
print("🔍 문제점 평가 시작...")


# 평가 결과 리스트 초기화
all_evaluated_results = []

# 후보 문제점 리스트를 하나씩 평가
for problem in tqdm(problem_candidates_for_evaluation):
    try:
        evaluated_result = evaluate_problems_with_rag(
            problem_candidates=[problem],  # 단일 문제점 리스트로 감쌈
            analysis_target_text_full=analysis_target_text_full,
            previous_full_summary_text=previous_full_summary_text,
            rag_vectorstore=rag_vectorstore,  # 사전 로드된 벡터스토어
            model="gpt-4o-2024-05-13"
        )
        all_evaluated_results.extend(evaluated_result)
    except Exception as e:
        print(f"⚠️ 문제점 평가 실패 (ID: {problem.get('id')}): {e}")

# 결과 저장 파일명 자동 생성
base_name = os.path.basename(summary_file_path).replace("_summary.txt", "")
save_path = os.path.join(save_folder, f"evaluation_{base_name}.json")

# 저장
with open(save_path, "w", encoding="utf-8") as f:
    json.dump(final_evaluation_results, f, indent=2, ensure_ascii=False)

print(f"\n✅ 평가 완료! 총 {len(final_evaluation_results)}개 문제점 평가 결과 저장됨")
print(f"📁 저장 위치: {save_path}")

🔍 문제점 평가 시작...


  0%|          | 0/669 [00:00<?, ?it/s]

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
평가 관련 참고 자료 검색 완료. 총 3개의 참고 자료 확보.

--- GPT 문제점 평가 시작 (ToT 평가) ---
평가 결과:
- 요약: 장해 평가 시 의료처치의 필수성: 총점 21점 (심각성: 8, 가능성: 7, 모호성: 6) - 이유: 약관에서 '장기이식을 하지 않고서는 생명유지가 불가능하여 의료처치를 평생토록 받아야 할 때'만을 흉복부장기 기능 상실로 인정하는 것은 많은 만성질환 환자들에게 실질적인 불이익을 초래할 수 있습니다. '만성질환 관리와 보험 혜택 p.145'에서 언급된 사례와 같이, 많은 만성질환 환자들이 의료처치를 통해 생명을 유지할 수 있는 상황이 현실적으로 더 많습니다. 따라서 이 기준은 소비자에게 큰 피해를 줄 수 있으며, 실제 발생 가능성도 높습니다. 약관 표현이 다소 모호하여 여러 해석이 가능하므로, 이로 인한 분쟁 가능성도 존재합니다.

--- [Step 3] 잠재적 문제점 평가 (1개 후보 대상, RAG 활용 가능) ---

--- 평가 대상 문제점 관련 약관 문구 검색 시작 ---
관련 약관 문구 검색 완료. 총 5개의 약관 청크 확보.

--- 평가 관련 참고 자료 검색 시작 ---
평가 관련 참고 자료 검색 완료. 총 3개의 참고 자료 확보.

--- GPT 문제점 평가 시작 (ToT 평가) ---
평가 결과:
- 요약: 일시적인 기능장해 제외 모호성: 총점 24점 (심각성: 8, 가능성: 7, 모호성: 9) - 이유: 약관에서 '관절을 사용하지 않아 발생한 일시적인 기능장해'는 보상에서 제외된다고 명시되어 있지만, 일시적인 장해가 지속될 경우에 대한 구체적인 기준이 모호합니다. 이는 소비자가 실질적인 보상을 받지 못할 위험이 큽니다. 추가 참고 자료에서 유사한 사례가 다수 존재하여 실제 발생 가능성도 높습니다. 약관 문구의 모호성으로 인해 소비자가 보험금을 청구할 때 분쟁이 발생할 가능성이 큽니다.

--- [Step 3] 잠재적 문제점 평가 (1개 후보

In [ ]:
# 결과 저장 파일명 자동 생성
base_name = os.path.basename(summary_file_path).replace("_summary.txt", "")
save_path = os.path.join(save_folder, f"evaluation_{base_name}.json")

# 저장
with open(save_path, "w", encoding="utf-8") as f:
    json.dump(all_evaluated_results, f, indent=2, ensure_ascii=False)

print(f"\n✅ 평가 완료! 총 {len(all_evaluated_results)}개 문제점 평가 결과 저장됨")
print(f"📁 저장 위치: {save_path}")


✅ 평가 완료! 총 669개 문제점 평가 결과 저장됨
📁 저장 위치: /content/drive/MyDrive/PROJECT/cn_api_project/step3_results/evaluation_한화_스마일펫보험.json


#### 함수 실행

In [ ]:
def batch_problem_evaluation(
    problem_candidates: list,
    analysis_target_text_full: str,  # RAG 전용, 직접 넣지 않음
    previous_full_summary_text: str,
    rag_vectorstore,
    batch_size: int = 20
):
    all_results = []
    for i in range(0, len(problem_candidates), batch_size):
        batch = problem_candidates[i:i+batch_size]
        try:
            results = evaluate_problems_with_rag(
                problem_candidates=batch,
                analysis_target_text_full="",  # 직접 넣지 않음 (벡터스토어에서만 검색)
                previous_full_summary_text=previous_full_summary_text,
                rag_vectorstore=rag_vectorstore
            )
            all_results.extend(results)
        except Exception as e:
            print(f"❌ 오류 발생 (batch {i}~{i+batch_size}): {e}")
    return all_results

In [ ]:
# 약관 원문 전체 및 이전 요약 텍스트 변수도 로드 코드 실행 후 정의되어 있어야 합니다.
analysis_target_text_full = locals().get('analysis_target_text_full', '')
previous_full_summary_text = locals().get('previous_full_summary_text', '')

In [ ]:
# 평가 실행 조건 확인: 평가할 후보 목록이 비어있지 않고, 벡터 스토어/GPT 모델/입력 텍스트가 로드되었다면 실행
if problem_candidates_for_evaluation and rag_vectorstore is not None and llm is not None and analysis_target_text_full and previous_full_summary_text:
    print(f"\n--- Step 3 잠재적 문제점 평가 시작 ({len(problem_candidates_for_evaluation)}개 후보 대상) ---")

    # evaluate_problems_with_rag 함수 호출하여 3단계 실행
    # 로드된 문제점 후보 리스트, 약관 텍스트 전체, 이전 요약 전체, 벡터 스토어 전달
    evaluated_problem_results = evaluate_problems_with_rag(
        problem_candidates=problem_candidates_for_evaluation, # 평가 대상 문제점 후보 목록 전달
        analysis_target_text_full=analysis_target_text_full, # 약관 원문 전체 전달
        previous_full_summary_text=previous_full_summary_text, # 이전 요약 전체 전달
        rag_vectorstore=rag_vectorstore, # 로드된 벡터 스토어 전달 (평가 시 RAG 활용)
        model="gpt-4o-2024-05-13" # GPT 모델
    )

    print("\n\n===== Step 3 최종 결과 (문제점 평가 및 중요 문제 선정) =====")

    # 가장 점수 높은 문제점 선정 로직
    top_problem = None # 초기화
    if evaluated_problem_results:
        sorted_evaluated_problems = sorted(evaluated_problem_results, key=lambda x: x.get('총점', 0), reverse=True)
        top_problem = sorted_evaluated_problems[0] if sorted_evaluated_problems and sorted_evaluated_problems[0].get('총점', -999) > -900 else None

    if top_problem:
        print(f"\n--- Step 3 선정된 가장 중요한 문제점 ---")
        # f-string 오류 방지
        근거_CoT_display_top = top_problem.get('근거_및_추론과정', 'N/A')[:150].replace('\n', ' ')
        평가_이유_display_top = top_problem.get('평가_이유_요약', 'N/A')
        print(f"요약: {top_problem.get('문제점_요약', 'N/A')}")
        print(f"총점: {top_problem.get('총점', 'N/A')}")
        print(f"근거 (CoT): {근거_CoT_display_top}...")
        print(f"평가 이유: {평가_이유_display_top}")
        print(f"관련 약관/자료: {top_problem.get('관련_약관_원문_표현', 'N/A')} | 출처: {top_problem.get('참고_자료_출처', 'N/A')}")
        # top_problem 변수에 가장 중요한 문제점 상세 내용이 담깁니다.
        # 이 변수를 다음 단계(Step 4: 개선 방안 발상)의 입력으로 사용합니다.
    else:
        print("평가 결과가 비어있거나 유효한 문제점이 없어 가장 중요한 문제점을 선정할 수 없습니다.")


elif not problem_candidates_for_evaluation:
    print("\n--- Step 3 실행 불가: Step 2 결과 (문제점 후보 목록)가 로드되지 않았습니다. ---")
    # 로드 경로 메시지는 이미 로드 코드에서 출력됩니다.
elif rag_vectorstore is None:
    print("\n--- Step 3 실행 불가: 벡터 스토어가 로드되지 않았습니다. ---")
    # 로드 경로 메시지는 이미 로드 코드에서 출력됩니다.
elif llm is None:
    print("\n--- Step 3 실행 불가: GPT 모델 로드 오류가 발생했습니다. ---")
    # 오류 메시지는 이미 로드 코드에서 출력됩니다.
elif not analysis_target_text_full or not previous_full_summary_text:
     print("\n--- Step 3 실행 불가: 분석 대상 약관 원문 또는 이전 약관 요약 텍스트가 비어있습니다. ---")



--- Step 3 잠재적 문제점 평가 시작 (669개 후보 대상) ---

--- [Step 3] 잠재적 문제점 평가 (669개 후보 대상, RAG 활용 가능) ---

--- 평가 대상 문제점 관련 약관 문구 검색 시작 ---
관련 약관 문구 검색 완료. 총 5개의 약관 청크 확보.

--- 평가 관련 참고 자료 검색 시작 ---
평가 관련 참고 자료 검색 완료. 총 3개의 참고 자료 확보.

--- GPT 문제점 평가 시작 (ToT 평가) ---
API 호출 중 오류 발생: Error code: 400 - {'error': {'message': "This model's maximum context length is 128000 tokens. However, your messages resulted in 233568 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}
!!! 문제점 평가 결과가 비어있습니다. 또는 응답 형식이 올바르지 않습니다. !!!
Raw Response Sample: {}...


===== Step 3 최종 결과 (문제점 평가 및 중요 문제 선정) =====
평가 결과가 비어있거나 유효한 문제점이 없어 가장 중요한 문제점을 선정할 수 없습니다.
